In [3]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords # Import the stop word list
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('comments.csv')

In [5]:
df.shape

(17885, 2)

In [6]:
df.sample(5)

,body,subreddit
15991,No.,nba
7542,is it the livestream?,rockets
17004,knicks bad,nba
14915,This was happening wayyyy before AT&amp;T boug...,nba
14619,Dub's defense is going to take a nosedive this...,nba


### Target

In [6]:
df['target'] = df['subreddit'].map({'nba': 0, 'rockets': 1})
df.drop('subreddit', axis=1, inplace=True)
df.sample(5)

,body,target
9558,Troll,0
3955,Are you sure the news you read on western medi...,1
7675,"Every year it gets worse, too. This year, Kawh...",1
4727,Crazy how james comes into the game and our wh...,1
9797,"No I didn't, because Wade did it too. The vict...",0


### Duplicates

In [7]:
# no na values in dataframe

df.isna().sum()

body      0
target    0
dtype: int64

In [8]:
# checking how many duplicate comments there are
df.duplicated().sum()

1336

In [9]:
# most of the duplicate comments are comments that were removed 
# duplicate comments were ether removed comments or comments that were posted twice with same text 
df[df.duplicated()].head(10)

,body,target
13,[removed],1
41,Says the dumbass,1
63,[removed],1
72,I had to unfollow Lebron on everything. I thi...,1
73,I had to unfollow Lebron on everything. I thi...,1
83,[removed],1
84,[removed],1
92,[removed],1
96,[removed],1
97,[removed],1


In [10]:
#drop rows where there are duplicates
df.drop_duplicates(inplace=True)
df = df.reset_index(drop=True)

In [11]:
df.shape

(16549, 2)

In [12]:
# drop rows where body = ''
df = df[df['body'] != '']
df = df.reset_index(drop=True)

In [13]:
df.shape

(16549, 2)

### Clean

In [14]:
df['body'][2]

"The change with China is going to happen when its economic bubble eventually bursts.\n\nThis is probably the best short breakdown of the Chinese cultural psyche that I've seen online:\n\nhttps://www.resetera.com/threads/rockets-gm-daryl-morey-tweets-in-support-of-hong-kong-protest-rockets-organization-denounces-tweet-update-nba-suspended-in-china.145209/page-13#post-25281281"

In [15]:
from bs4 import BeautifulSoup             

# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(df['body'][2])

# Print the raw review and then the output of get_text(), for 
# comparison
print(df['body'][2])
print()
print(example1.get_text())

The change with China is going to happen when its economic bubble eventually bursts.

This is probably the best short breakdown of the Chinese cultural psyche that I've seen online:

https://www.resetera.com/threads/rockets-gm-daryl-morey-tweets-in-support-of-hong-kong-protest-rockets-organization-denounces-tweet-update-nba-suspended-in-china.145209/page-13#post-25281281

The change with China is going to happen when its economic bubble eventually bursts.

This is probably the best short breakdown of the Chinese cultural psyche that I've seen online:

https://www.resetera.com/threads/rockets-gm-daryl-morey-tweets-in-support-of-hong-kong-protest-rockets-organization-denounces-tweet-update-nba-suspended-in-china.145209/page-13#post-25281281


In [16]:
# Use regular expressions to do a find-and-replace
letters_only = re.sub("[^a-zA-Z]",           # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text())   # The text to search

In [17]:
# Convert letters_only to lower case.
lower_case = letters_only.lower()

# Split lower_case up at each space.
words = lower_case.split() # This is like a manual tokenizer!

In [18]:
# Check first ten words.
words[0:10]

['the',
 'change',
 'with',
 'china',
 'is',
 'going',
 'to',
 'happen',
 'when',
 'its']

In [19]:
# Funtion adapted from Lecture 

def review_to_words(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove HTML.
    review_text = BeautifulSoup(text).get_text()

    # Remove HTML special entities (e.g. &amp;)
    text = re.sub(r'\&\w*;', '', text)
    
    # Remove non-letters. (Punctuation)
    text = re.sub("[^a-zA-Z]", " ", text)
    
    # Remove hyperlinks
    text = re.sub(r'https?:\/\/.*\/\w*', '', text)
    
    # Remove words with 2 or fewer letters
    text = re.sub(r'\b\w{1,2}\b', '', text)
    
    # Remove whitespaces
    text = re.sub(r'\s\s+', ' ', text)
     
    return text

In [20]:
df['body'] = df['body'].apply(review_to_words)

/home/dylan/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://www.youtube.com/watch?v=qb2aimerktc" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/dylan/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://imgur.com/paehnd0" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/dylan/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://imgur.com/kurdwus" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that doc

/home/dylan/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://www.google.com/amp/s/amp.cnn.com/cnn/2019/10/06/asia/china-xinjiang-video-intl-hnk/index.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/dylan/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://twitter.com/shamscharania/status/1183078170947395586" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/home/dylan/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:357: UserWarning: "https://imgur.com/gallery/o1cn9gx" looks like a URL. Beautiful Soup is not an HTTP client. You s

In [21]:
df.shape

(16549, 2)

In [22]:
df['body'][2]

'the change with china going happen when its economic bubble eventually bursts this probably the best short breakdown the chinese cultural psyche that seen online https www resetera com threads rockets daryl morey tweets support hong kong protest rockets organization denounces tweet update nba suspended china page post '

### Lemmatize

In [23]:
lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    words = text.split()
    lemma_words = ''
    for word in words:
        lemma_words += (lemmatizer.lemmatize(word) + ' ')
    return lemma_words   

In [24]:
df['body'] = df['body'].apply(lemmatize_words)

In [25]:
df.shape

(16549, 2)

In [26]:
df.to_csv('clean_comments.csv', index=False)

### Find most common words from each subreddit

In [85]:
# Get most frequently used words, can change max_features

count_vect = CountVectorizer(analyzer = "word", 
                             tokenizer = None, 
                             preprocessor = None,
                             stop_words = "english", 
                             max_features = None) 

# input for CountVectorizer is an array of strings
vector_input_rockets = df[df['target'] == 1]['body']

# fit_transform the vectorizer
rockets_words = count_vect.fit_transform(vector_input_rockets)

# convert output to a Numpy array
rockets_words = rockets_words.toarray()

In [86]:
# rockets most common words
rockets_word_list = count_vect.get_feature_names()
print(rockets_word_list)

['aaaaand', 'aadldwh', 'aaha', 'aahhhh', 'aaron', 'ab', 'abandoned', 'abc', 'abeard', 'abet', 'abide', 'abiding', 'ability', 'able', 'aboard', 'abolish', 'aboutism', 'abouve', 'abraham', 'abrasive', 'abroad', 'absence', 'absentmindedly', 'absolute', 'absolutely', 'absorbed', 'absoslute', 'abstain', 'absurb', 'absurd', 'absurdity', 'abt', 'abuse', 'abused', 'abusive', 'academia', 'academic', 'academy', 'acc', 'accelerates', 'accent', 'accentuation', 'accept', 'acceptable', 'accepted', 'access', 'accidentally', 'accommodate', 'accommodating', 'accomplish', 'accomplished', 'accomplishes', 'according', 'accordingly', 'account', 'accountable', 'accounted', 'accumulated', 'accurate', 'accurately', 'accuse', 'accusing', 'achieve', 'achieved', 'acknowledged', 'acknowledging', 'acl', 'acquaintance', 'acquire', 'acquired', 'acrimony', 'act', 'acted', 'acting', 'action', 'activate', 'active', 'actively', 'activism', 'activist', 'activity', 'actor', 'actress', 'actual', 'actually', 'ad', 'adam', '

In [87]:
# Get most frequently used words, can change max_features

count_vect = CountVectorizer(analyzer = "word", 
                             tokenizer = None, 
                             preprocessor = None,
                             stop_words = "english", 
                             max_features = None) 

# input for CountVectorizer is an array of strings
vector_input_nba = df[df['target'] == 0]['body']

# fit_transform the vectorizer
nba_words = count_vect.fit_transform(vector_input_nba)

# convert output to a Numpy array
nba_words = nba_words.toarray()

In [88]:
# nba most common words
nba_word_list = count_vect.get_feature_names()
print(nba_word_list)

['aaaand', 'aaaye', 'aand', 'aaron', 'aarp', 'aau', 'abandon', 'abandoned', 'abbreviate', 'abc', 'abcnews', 'abducted', 'abdul', 'abhorrent', 'abide', 'ability', 'able', 'abnormal', 'abnormally', 'aboard', 'aboit', 'abomination', 'abortion', 'abotu', 'aboutism', 'abraham', 'abridging', 'abroad', 'absence', 'absent', 'absoloutely', 'absolute', 'absolutely', 'absolutist', 'absolutley', 'absolve', 'absorbed', 'abstain', 'abstinence', 'abstract', 'absurd', 'abuse', 'abused', 'abuser', 'abusing', 'abusive', 'abyss', 'academia', 'academic', 'academy', 'accent', 'accept', 'acceptable', 'accepted', 'accepting', 'access', 'accessory', 'accident', 'accidental', 'accidentally', 'accolade', 'accommodate', 'accommodated', 'accompanied', 'accomplised', 'accomplish', 'accomplished', 'accomplishes', 'accomplishment', 'accord', 'accordance', 'according', 'accordingly', 'account', 'accountable', 'accounting', 'acct', 'accuracy', 'accurate', 'accurately', 'accusation', 'accuse', 'accused', 'accusing', 'a

### Add the most common words that are in both subreddits to stopwords 'english'

In [89]:
# Function to find elements that are the same in two lists 

def common_words(a, b): 
    a_set = set(a) 
    b_set = set(b) 
    if (a_set & b_set): 
        print(a_set & b_set) 
    else: 
        print("No common elements")  

In [90]:
common_words(rockets_word_list, nba_word_list)

{'return', 'modern', 'compare', 'quick', 'sportnet', 'rookie', 'giant', 'filler', 'quickness', 'frazier', 'wisdom', 'lately', 'wilt', 'mainly', 'russian', 'emotionally', 'hotel', 'rudy', 'demolish', 'lottery', 'mainlanders', 'achieve', 'wayyy', 'aggression', 'solely', 'ceiling', 'shop', 'quo', 'disagreement', 'coming', 'politically', 'dieng', 'paste', 'lifting', 'sourced', 'breakdown', 'grey', 'capital', 'scramble', 'derozan', 'ehhh', 'sent', 'iossmf', 'heave', 'green', 'audience', 'guarantee', 'net', 'repping', 'shorter', 'squeezing', 'codified', 'genetic', 'trey', 'rolled', 'shitshow', 'northern', 'incorporate', 'screenshot', 'cancelled', 'permanently', 'dick', 'construct', 'trouble', 'internment', 'returned', 'vote', 'vince', 'translated', 'apologist', 'hongkongfp', 'ranger', 'obligatory', 'fwiw', 'told', 'touched', 'moreys', 'indicated', 'deference', 'relation', 'criticism', 'deserves', 'outta', 'censored', 'clinton', 'temporarily', 'io', 'angry', 'magnitude', 'memorabilia', 'repla

In [91]:
union_words = ['return', 'modern', 'compare', 'quick', 'sportnet', 'rookie', 'giant', 'filler', 'quickness', 'frazier', 'wisdom', 'lately', 'wilt', 'mainly', 'russian', 'emotionally', 'hotel', 'rudy', 'demolish', 'lottery', 'mainlanders', 'achieve', 'wayyy', 'aggression', 'solely', 'ceiling', 'shop', 'quo', 'disagreement', 'coming', 'politically', 'dieng', 'paste', 'lifting', 'sourced', 'breakdown', 'grey', 'capital', 'scramble', 'derozan', 'ehhh', 'sent', 'iossmf', 'heave', 'green', 'audience', 'guarantee', 'net', 'repping', 'shorter', 'squeezing', 'codified', 'genetic', 'trey', 'rolled', 'shitshow', 'northern', 'incorporate', 'screenshot', 'cancelled', 'permanently', 'dick', 'construct', 'trouble', 'internment', 'returned', 'vote', 'vince', 'translated', 'apologist', 'hongkongfp', 'ranger', 'obligatory', 'fwiw', 'told', 'touched', 'moreys', 'indicated', 'deference', 'relation', 'criticism', 'deserves', 'outta', 'censored', 'clinton', 'temporarily', 'io', 'angry', 'magnitude', 'memorabilia', 'replace', 'quinn', 'sun', 'satisfied', 'established', 'disgrace', 'protested', 'tagged', 'worth', 'wearing', 'extended', 'kim', 'bro', 'variable', 'redditors', 'playing', 'heavy', 'debuting', 'fastest', 'proof', 'mixed', 'scorer', 'disturbing', 'participate', 'incompetent', 'ariza', 'contract', 'asking', 'past', 'realise', 'downward', 'espn', 'forge', 'qualifier', 'regardless', 'reportedly', 'immigration', 'porn', 'holocaust', 'heartbreaking', 'answering', 'make', 'integral', 'overlap', 'roland', 'accurate', 'quiet', 'decision', 'homo', 'mainstream', 'iraq', 'died', 'backlash', 'emmy', 'vorp', 'called', 'strive', 'authority', 'personified', 'september', 'departure', 'genuine', 'optimistic', 'location', 'beef', 'stupidity', 'ruined', 'practice', 'stock', 'roasting', 'stats', 'denied', 'ruin', 'hedge', 'rally', 'crunch', 'group', 'personnel', 'wore', 'worn', 'mood', 'simmons', 'morally', 'greece', 'lowry', 'scenery', 'shot', 'remained', 'feared', 'team', 'sympathy', 'private', 'capitol', 'operate', 'fade', 'goal', 'fox', 'syria', 'usually', 'serving', 'breaker', 'mvp', 'asian', 'edited', 'snake', 'controversial', 'planning', 'lose', 'currently', 'overriding', 'discussed', 'versus', 'protesting', 'better', 'headache', 'fourth', 'strange', 'laughing', 'sitting', 'fan', 'employed', 'theme', 'alleged', 'lowering', 'reggie', 'ancestor', 'info', 'week', 'thunder', 'bam', 'middle', 'originally', 'center', 'terrorist', 'denounce', 'direction', 'profit', 'involved', 'flip', 'firm', 'historic', 'foolish', 'conclusion', 'taken', 'state', 'discouraged', 'clyde', 'far', 'uncertain', 'conversion', 'breather', 'year', 'han', 'unwilling', 'lopsided', 'intro', 'undersized', 'ruled', 'sword', 'click', 'unwatchable', 'mitchell', 'percentage', 'china', 'jealous', 'rep', 'ain', 'throw', 'commenters', 'academic', 'hustle', 'hahaha', 'howard', 'negatively', 'fun', 'lost', 'customer', 'retire', 'normally', 'certainty', 'mao', 'feigen', 'mistake', 'important', 'adamsilverfanpage', 'sus', 'attempt', 'gym', 'weak', 'assume', 'national', 'controversy', 'event', 'cfa', 'reflect', 'offended', 'fee', 'bash', 'anxiety', 'federal', 'freely', 'incarceration', 'statlines', 'cash', 'doe', 'explaining', 'historian', 'slight', 'lowe', 'regarding', 'japan', 'result', 'wiped', 'absolute', 'aboutism', 'career', 'ocean', 'assumed', 'safe', 'disappearing', 'elected', 'cousin', 'recruiting', 'championship', 'border', 'remain', 'airport', 'pregame', 'preach', 'physicality', 'characterized', 'chucking', 'carried', 'accommodate', 'january', 'colony', 'listening', 'reversing', 'discredit', 'july', 'lakeshow', 'woah', 'saving', 'park', 'debut', 'conditioning', 'thrive', 'ideology', 'stud', 'true', 'tbf', 'iteration', 'bunch', 'subtle', 'notoriously', 'supposed', 'squeeze', 'involves', 'unemployed', 'suppose', 'youtube', 'shaquille', 'substance', 'highest', 'damned', 'consistently', 'compass', 'volunteer', 'ray', 'owned', 'engaging', 'admitting', 'luck', 'fantastic', 'agreed', 'ibaka', 'refund', 'lal', 'boring', 'include', 'harvest', 'switching', 'merch', 'facility', 'hottest', 'apologized', 'identical', 'brilliant', 'legally', 'claimed', 'fall', 'carve', 'rocket', 'ago', 'suspended', 'correctly', 'human', 'rebuke', 'operates', 'stress', 'later', 'flew', 'reduce', 'hiding', 'communicate', 'deal', 'dipshit', 'escalation', 'fucken', 'disappeared', 'preserve', 'ref', 'irrational', 'criticise', 'looked', 'differential', 'improve', 'trusted', 'latest', 'edu', 'relevance', 'behave', 'talk', 'brady', 'promoting', 'nasty', 'thicker', 'hater', 'podcast', 'incredible', 'manages', 'beginning', 'steven', 'pickup', 'corrupt', 'boot', 'involving', 'toxic', 'asks', 'kendrick', 'bothered', 'merry', 'attitude', 'condemned', 'acceptable', 'worked', 'destroy', 'disappearance', 'example', 'lack', 'storm', 'intelligence', 'regional', 'superstar', 'turkish', 'performed', 'reaction', 'bodyguard', 'exception', 'acquire', 'injured', 'dub', 'beta', 'pm', 'brutal', 'spit', 'yall', 'forward', 'slippery', 'appeared', 'patch', 'traditionally', 'virtue', 'bold', 'landing', 'gun', 'chamber', 'jet', 'treatment', 'answer', 'grows', 'underestimate', 'sold', 'viacom', 'catalyst', 'medium', 'correct', 'edition', 'objectively', 'skilled', 'whew', 'substantial', 'arizona', 'mcdonald', 'wont', 'festival', 'hahahahah', 'proper', 'confusing', 'subscribed', 'allstar', 'brutality', 'describes', 'approved', 'balance', 'streisand', 'clearly', 'defensively', 'positive', 'implying', 'highschool', 'screw', 'calling', 'filled', 'turkey', 'authoritarianism', 'capped', 'outright', 'papa', 'investor', 'escape', 'seizure', 'era', 'murdered', 'revenge', 'lead', 'compton', 'health', 'rubber', 'xenophobic', 'submit', 'robbery', 'spark', 'oliver', 'replicate', 'shoulder', 'timely', 'success', 'praying', 'shocked', 'complicated', 'robbed', 'aging', 'lining', 'study', 'threaten', 'attention', 'hongkongers', 'independent', 'averted', 'laughable', 'guantanamo', 'radar', 'definition', 'store', 'colour', 'realizes', 'maturing', 'fluffy', 'holder', 'bigs', 'asshole', 'separatism', 'mchale', 'hungry', 'dragging', 'california', 'orleans', 'portion', 'litmus', 'homophobic', 'lure', 'viewed', 'nazi', 'carter', 'dynasty', 'flex', 'lowered', 'disadvantage', 'contact', 'dont', 'abstain', 'aged', 'danny', 'utm', 'slightest', 'specimen', 'choice', 'kawhis', 'hou', 'nervous', 'count', 'reliable', 'telling', 'www', 'grammatically', 'hated', 'homer', 'woj', 'offered', 'develop', 'fired', 'rape', 'gaining', 'started', 'offer', 'including', 'blocking', 'suspected', 'choked', 'power', 'derrick', 'guardable', 'travel', 'stupid', 'earn', 'wayyyy', 'pound', 'limping', 'legitimate', 'hesitant', 'handoff', 'migrant', 'balled', 'improved', 'dominate', 'violating', 'bang', 'finished', 'ppl', 'spread', 'harden', 'redemption', 'succeed', 'spacing', 'grow', 'smile', 'morris', 'accent', 'cock', 'loss', 'message', 'control', 'sake', 'unrelated', 'pardon', 'avoiding', 'montrezl', 'notice', 'newly', 'subreddit', 'jerk', 'surely', 'alibaba', 'phenomenal', 'dwight', 'arguable', 'league', 'apartment', 'pointer', 'hole', 'colleague', 'religious', 'successor', 'limit', 'badly', 'changed', 'love', 'iffy', 'neutral', 'rural', 'truly', 'carved', 'usage', 'bail', 'ugly', 'nyc', 'transfer', 'mile', 'violence', 'dunker', 'tellin', 'blueprint', 'alienating', 'musician', 'fake', 'cuban', 'madison', 'cctv', 'bricking', 'arent', 'hella', 'dicked', 'strategically', 'bread', 'socal', 'draymond', 'legendary', 'induced', 'sticking', 'entitlement', 'letting', 'florida', 'arguing', 'pulled', 'june', 'nationalist', 'mindset', 'winning', 'er', 'list', 'lmaooo', 'figure', 'sterling', 'resulted', 'reasonable', 'consent', 'feeding', 'realizing', 'trolling', 'googling', 'miserable', 'scheduled', 'answered', 'helpful', 'motherfuckin', 'reuters', 'alter', 'rondo', 'world', 'fascinating', 'gotten', 'immediately', 'mid', 'obnoxious', 'successful', 'protest', 'happy', 'subway', 'ranked', 'cu', 'cheese', 'gear', 'froze', 'civilian', 'defend', 'repeated', 'necessarily', 'ice', 'salt', 'odds', 'equivalence', 'thicc', 'noted', 'want', 'local', 'rush', 'forgets', 'bathe', 'wild', 'imprisoned', 'bring', 'reg', 'salty', 'strongly', 'money', 'dropped', 'illegally', 'eastern', 'firestorm', 'bending', 'steal', 'ferguson', 'black', 'shed', 'drinking', 'refrain', 'calculated', 'dialogue', 'replacement', 'eric', 'cancer', 'google', 'flustered', 'gang', 'majority', 'litty', 'shove', 'match', 'wire', 'amare', 'session', 'fault', 'arrested', 'liu', 'vision', 'head', 'buff', 'celebration', 'anchor', 'deserved', 'size', 'suffers', 'hasn', 'interaction', 'dis', 'surveillance', 'depends', 'root', 'unstoppable', 'intense', 'reveal', 'competitive', 'agree', 'capacity', 'wonderful', 'indirectly', 'typing', 'bus', 'limited', 'fails', 'applaud', 'test', 'yell', 'cross', 'spelling', 'figuratively', 'laying', 'tight', 'idk', 'multi', 'chriss', 'moot', 'loses', 'harsh', 'convinced', 'reddits', 'yelling', 'lip', 'starve', 'link', 'tom', 'wizard', 'expanded', 'monitoring', 'temper', 'painting', 'asf', 'bean', 'guarded', 'real', 'excellent', 'dumbass', 'union', 'whoa', 'britain', 'slam', 'hypocritical', 'tim', 'highlight', 'caused', 'read', 'development', 'heavier', 'congrats', 'colonial', 'fair', 'context', 'star', 'environment', 'obtuse', 'bbq', 'tolerate', 'esque', 'irl', 'leg', 'directly', 'infringe', 'tariff', 'phase', 'personally', 'adversity', 'greatness', 'india', 'nbas', 'inject', 'anytime', 'man', 'inning', 'suited', 'minimal', 'judge', 'encourage', 'lamelo', 'barely', 'specialized', 'stick', 'tends', 'believed', 'began', 'famer', 'web', 'str', 'enact', 'smoothly', 'reverse', 'shown', 'alike', 'betterment', 'protester', 'cheering', 'leaf', 'daryl', 'knowledge', 'fined', 'attack', 'literal', 'offend', 'applied', 'perceive', 'outlier', 'expanding', 'snowballing', 'lowkey', 'ton', 'potato', 'tucker', 'listen', 'referring', 'proving', 'pls', 'food', 'arc', 'roast', 'savage', 'dems', 'dog', 'chemistry', 'responded', 'excessive', 'feb', 'thats', 'tweeting', 'talking', 'woke', 'switched', 'asg', 'critique', 'station', 'valuable', 'pawn', 'screwed', 'cute', 'aspect', 'nope', 'instagram', 'hur', 'apply', 'throwback', 'inclined', 'gather', 'representative', 'raw', 'specifically', 'trailing', 'hardens', 'capitalist', 'lion', 'underrated', 'congress', 'constant', 'grandpa', 'threat', 'invaded', 'life', 'avoid', 'judging', 'band', 'imagined', 'biggest', 'came', 'family', 'apology', 'whomever', 'chef', 'bouncing', 'insinuating', 'official', 'negro', 'earlier', 'hyping', 'smacked', 'collapse', 'adding', 'slogan', 'clarity', 'variety', 'sorta', 'ayy', 'gotchu', 'confused', 'rpg', 'global', 'dpoy', 'influence', 'kevin', 'extent', 'freak', 'resell', 'appreciative', 'vibe', 'pump', 'climate', 'spray', 'mandatory', 'kong', 'giphy', 'fragile', 'oracle', 'sticker', 'chubby', 'japanese', 'dude', 'non', 'amazed', 'rapist', 'sham', 'financially', 'collective', 'mind', 'ratio', 'air', 'regard', 'rehab', 'angle', 'rioter', 'ballot', 'forced', 'exciting', 'russell', 'rob', 'alley', 'replay', 'regular', 'towel', 'copy', 'refusing', 'hero', 'inch', 'dang', 'tendency', 'optimism', 'sneaker', 'uni', 'boomer', 'celebs', 'brad', 'iman', 'horford', 'charged', 'thug', 'inefficient', 'exhibition', 'africa', 'blink', 'face', 'weight', 'impression', 'sincerely', 'uncontested', 'swinging', 'aesthetic', 'resulting', 'excusing', 'lever', 'irritates', 'accusing', 'mountain', 'fouling', 'kat', 'okay', 'language', 'wishful', 'whilst', 'fringe', 'mechanic', 'louder', 'risky', 'forcing', 'apart', 'ming', 'preseason', 'chess', 'occupying', 'marketing', 'carmelo', 'statistic', 'device', 'declined', 'attractive', 'varsity', 'converted', 'firing', 'inevitably', 'fucking', 'professional', 'prepared', 'trip', 'extension', 'wasted', 'alternate', 'suffrage', 'lit', 'gonna', 'fix', 'surgery', 'comparison', 'knew', 'town', 'agent', 'billion', 'sniffing', 'gone', 'enlightened', 'stretch', 'contributor', 'mccaw', 'flexibility', 'spewing', 'peripheral', 'fortnite', 'remaining', 'afraid', 'markelle', 'misunderstanding', 'gain', 'businessmen', 'signature', 'qing', 'historically', 'democratic', 'raymond', 'pierce', 'knee', 'stake', 'yao', 'unacceptable', 'layer', 'canadian', 'factory', 'north', 'work', 'sarcasm', 'unable', 'uncomfortable', 'grainy', 'highly', 'negotiate', 'hire', 'clipper', 'tillman', 'cost', 'candidate', 'rat', 'advertise', 'agreement', 'meaningful', 'recognizes', 'nicest', 'limitation', 'certainly', 'malice', 'virtually', 'butthole', 'dumbest', 'wanna', 'simple', 'warranted', 'foot', 'williams', 'chapter', 'criticized', 'complex', 'lie', 'clock', 'bod', 'toronto', 'approach', 'minor', 'cody', 'barring', 'astronomical', 'building', 'opened', 'fracture', 'sending', 'couldve', 'pushing', 'stan', 'ennis', 'enjoy', 'admits', 'doubt', 'dunked', 'opposed', 'inconsistent', 'influenced', 'wealth', 'punishment', 'listened', 'disagree', 'actively', 'jamychal', 'oct', 'san', 'agrees', 'randy', 'enemy', 'billups', 'format', 'hollywood', 'inability', 'credit', 'davis', 'method', 'eating', 'complete', 'dancer', 'download', 'ppp', 'sacrificing', 'section', 'pointed', 'sure', 'opium', 'deflected', 'clean', 'fandom', 'visit', 'omg', 'swing', 'practicing', 'dislike', 'exactly', 'dispute', 'faced', 'kardashians', 'photo', 'county', 'taipei', 'pervasive', 'word', 'progressed', 'worse', 'motivation', 'loyalty', 'needlessly', 'maybe', 'affair', 'naked', 'conjecture', 'vastly', 'daily', 'athletic', 'careful', 'natural', 'landed', 'refused', 'aight', 'probably', 'really', 'triumph', 'beard', 'casual', 'tore', 'dribble', 'sum', 'sit', 'forever', 'christmas', 'demon', 'act', 'understood', 'shitty', 'detained', 'unanimously', 'driver', 'finesse', 'benefited', 'athletics', 'bracket', 'ended', 'redick', 'kowtowing', 'ouch', 'wealthy', 'index', 'nonetheless', 'booker', 'outweigh', 'grateful', 'mlb', 'interesting', 'terrified', 'nagging', 'helped', 'luka', 'best', 'blowing', 'exact', 'stigma', 'expires', 'throwaway', 'leave', 'refer', 'rage', 'quickly', 'floater', 'note', 'manner', 'pt', 'interference', 'twolves', 'birthday', 'laser', 'yep', 'mild', 'revision', 'lock', 'landry', 'notification', 'dense', 'designated', 'actually', 'crawford', 'november', 'tor', 'dictatorship', 'cringe', 'cheating', 'expecting', 'venture', 'empire', 'warned', 'ask', 'musical', 'bud', 'pooh', 'firsthand', 'drafting', 'happens', 'opinion', 'opposing', 'constructed', 'boxing', 'government', 'overblown', 'finally', 'unbiased', 'motor', 'takeover', 'boat', 'monetary', 'effort', 'bit', 'buzzer', 'uyghur', 'demonstrate', 'blasted', 'matter', 'flaming', 'dissuade', 'fast', 'banking', 'smartest', 'mortal', 'wondered', 'playoff', 'reacts', 'speak', 'charge', 'denies', 'score', 'embarrassing', 'responsibility', 'economics', 'demoralize', 'wrote', 'sparking', 'stone', 'ally', 'ultimate', 'conference', 'mud', 'swim', 'gordon', 'bridge', 'murder', 'charity', 'crazier', 'mileage', 'validate', 'cutter', 'perception', 'sigh', 'pimp', 'veteran', 'personality', 'hyperbolic', 'business', 'righteous', 'regret', 'rotten', 'hayes', 'intensity', 'won', 'lukewarm', 'effect', 'panda', 'elimination', 'collectively', 'rich', 'loud', 'pushed', 'atm', 'led', 'relative', 'gunning', 'prefer', 'reckon', 'homeboy', 'caring', 'easily', 'rarity', 'explicitly', 'oladipo', 'crowder', 'distance', 'chew', 'edward', 'discussion', 'damage', 'sovereignty', 'lac', 'delay', 'randomly', 'beer', 'related', 'cared', 'forbid', 'hour', 'quite', 'square', 'dip', 'equal', 'petition', 'lame', 'karma', 'bear', 'matt', 'dissing', 'absurd', 'debate', 'tested', 'hakeem', 'dwade', 'jumping', 'backcourt', 'advertisement', 'send', 'farm', 'observation', 'anime', 'violation', 'seek', 'immigrant', 'image', 'bay', 'crooked', 'wisely', 'cartoon', 'criterion', 'assistant', 'horrific', 'wing', 'tense', 'quadruple', 'uncanny', 'drive', 'dur', 'reducing', 'improves', 'projecting', 'lmaoooo', 'baiting', 'butler', 'necessary', 'owner', 'hart', 'bump', 'aware', 'pole', 'foul', 'button', 'tune', 'phrased', 'primary', 'fixing', 'comical', 'conflict', 'significantly', 'following', 'wasnt', 'addressed', 'visible', 'floor', 'outraged', 'belief', 'campaign', 'negative', 'org', 'european', 'ludicrous', 'demonstration', 'shuffle', 'broader', 'gas', 'tilmans', 'younger', 'sole', 'noah', 'million', 'piggybacking', 'citizenship', 'goalpost', 'biased', 'shout', 'mutual', 'writer', 'topic', 'likewise', 'twat', 'bosh', 'establish', 'retweeting', 'ncaa', 'globally', 'hadn', 'dismiss', 'rub', 'yikes', 'forgetting', 'stupidest', 'great', 'defense', 'parker', 'daniel', 'icon', 'fcomments', 'sleep', 'boston', 'mac', 'piss', 'stfu', 'higher', 'sea', 'dan', 'donald', 'shaved', 'routine', 'soul', 'reach', 'ridiculously', 'hongkonger', 'potus', 'target', 'university', 'wanted', 'release', 'fella', 'mysterious', 'kidnapping', 'linked', 'statistically', 'tyson', 'expect', 'forming', 'busy', 'judgement', 'killing', 'packed', 'evil', 'asset', 'cam', 'gigem', 'award', 'singing', 'poverty', 'suffer', 'resign', 'repeater', 'photoshop', 'spew', 'barea', 'finding', 'lying', 'decade', 'pizza', 'newest', 'precedent', 'income', 'education', 'thanked', 'maintaining', 'backing', 'posture', 'denver', 'bargain', 'fear', 'workout', 'cold', 'single', 'crash', 'pipe', 'triple', 'kardashian', 'jew', 'relate', 'mental', 'stocked', 'tied', 'bron', 'hate', 'politics', 'performing', 'incase', 'opener', 'owes', 'poorly', 'backed', 'courtside', 'protect', 'series', 'track', 'lmfao', 'kept', 'oppressive', 'drain', 'provided', 'whichever', 'comparable', 'orchestrating', 'laughed', 'sexy', 'durant', 'pure', 'iraqi', 'midrange', 'importantly', 'replying', 'financial', 'dying', 'fathom', 'mentally', 'paying', 'fart', 'slash', 'upper', 'beautiful', 'logic', 'venue', 'equate', 'stay', 'casino', 'sided', 'elton', 'stayed', 'confronted', 'create', 'solution', 'orlando', 'dribbling', 'hindsight', 'nightmare', 'defeat', 'empathy', 'hook', 'gray', 'blocked', 'atmosphere', 'showing', 'openly', 'duh', 'vpn', 'treaty', 'vernon', 'supporting', 'unusually', 'ohhhhh', 'incentive', 'thank', 'chief', 'warrior', 'pas', 'select', 'shall', 'failed', 'automatically', 'doncic', 'named', 'chest', 'row', 'sex', 'endless', 'military', 'sample', 'playmaking', 'skill', 'chosen', 'ashamed', 'italian', 'eye', 'hometown', 'deleting', 'monthly', 'continues', 'accordingly', 'step', 'shady', 'rightly', 'imperial', 'phd', 'tsai', 'took', 'praising', 'digest', 'principle', 'set', 'spent', 'pilot', 'crushed', 'abusive', 'wait', 'gotta', 'reddit', 'silenced', 'bashing', 'averaging', 'tyrant', 'clicked', 'express', 'remindermessage', 'skip', 'tennis', 'included', 'worthy', 'chick', 'fanboys', 'evolved', 'isaiah', 'bank', 'wrestler', 'newer', 'ironic', 'respond', 'centered', 'coast', 'bombarded', 'gasol', 'lane', 'dollar', 'kicked', 'malcolm', 'menu', 'throwing', 'anger', 'chair', 'stolen', 'australia', 'review', 'production', 'sturdy', 'appropriate', 'deflect', 'owns', 'rid', 'aid', 'slide', 'specialist', 'circle', 'wolf', 'iota', 'sheet', 'stopped', 'speculate', 'emergency', 'ive', 'audacity', 'closer', 'gm', 'barefoot', 'undermine', 'edit', 'spectacular', 'streak', 'cultural', 'incorporating', 'super', 'replied', 'incident', 'mute', 'apologize', 'let', 'profiting', 'aaron', 'ownership', 'plo', 'msnbc', 'content', 'hoping', 'snowflake', 'defending', 'concerning', 'durable', 'rex', 'setup', 'goddamn', 'hearing', 'math', 'nationally', 'experiment', 'released', 'deciding', 'purple', 'board', 'fresh', 'entire', 'likeable', 'kidding', 'recall', 'sighted', 'steak', 'effected', 'chart', 'shooting', 'simultaneously', 'rated', 'scheme', 'forehead', 'shared', 'moment', 'coached', 'ha', 'wade', 'sign', 'wake', 'slavery', 'planned', 'sexual', 'territorial', 'pain', 'slaughtering', 'rock', 'span', 'nelson', 'golf', 'infuriating', 'road', 'translates', 'scott', 'remains', 'chairman', 'drastic', 'title', 'conduct', 'pragmatic', 'beloved', 'jam', 'awkward', 'narcissistic', 'like', 'educated', 'populace', 'humble', 'miss', 'activism', 'consistency', 'actual', 'tattoo', 'waive', 'industry', 'backpedaling', 'reminded', 'rotating', 'unlikely', 'def', 'bend', 'echo', 'traded', 'fertita', 'subject', 'educate', 'men', 'male', 'wiz', 'enforcement', 'equality', 'mutually', 'lick', 'melo', 'continue', 'quoted', 'portis', 'viewing', 'hilariously', 'chip', 'nearly', 'foreigner', 'overcome', 'society', 'classy', 'agency', 'detrimental', 'river', 'stevens', 'worker', 'gallo', 'effective', 'total', 'patient', 'shitload', 'greater', 'provide', 'chose', 'blindly', 'slander', 'voting', 'podcasts', 'greedy', 'stacked', 'ignorance', 'forgotten', 'sucked', 'brook', 'vary', 'opportunity', 'body', 'consumption', 'welcome', 'comment', 'brainwashed', 'ben', 'competitor', 'outcry', 'halloween', 'realer', 'haul', 'tech', 'major', 'ric', 'nene', 'prison', 'ending', 'confirm', 'glad', 'donovan', 'alot', 'learned', 'emotional', 'bye', 'spoiled', 'porter', 'coincidence', 'grace', 'construction', 'nash', 'ketchup', 'critical', 'late', 'carsen', 'dunking', 'outstanding', 'popping', 'hideous', 'cmon', 'surpass', 'excitement', 'frustration', 'tag', 'dweller', 'disappointing', 'underpaid', 'term', 'checker', 'humor', 'nd', 'ringer', 'heard', 'astros', 'sync', 'inform', 'wesley', 'corey', 'didnt', 'execute', 'accomplished', 'eventually', 'start', 'massage', 'joined', 'alt', 'founded', 'woman', 'ease', 'rider', 'thankfully', 'toyota', 'explanation', 'kaepernick', 'favorite', 'fight', 'cavs', 'testament', 'respectful', 'cleared', 'remembered', 'site', 'directed', 'legal', 'indian', 'adjustment', 'went', 'spend', 'wiki', 'pissed', 'troy', 'occasional', 'accomplish', 'easy', 'excuse', 'shaking', 'bullying', 'reduced', 'small', 'weekday', 'appealing', 'transition', 'boi', 'measuring', 'walmart', 'wwii', 'run', 'weirder', 'relied', 'attacking', 'mall', 'representation', 'hosting', 'activist', 'russel', 'quote', 'socially', 'fought', 'harder', 'minimum', 'ainge', 'rozier', 'array', 'transplant', 'standard', 'respected', 'alongside', 'hack', 'brooklyn', 'affect', 'controlling', 'intelligent', 'missing', 'ignorant', 'duerte', 'practical', 'kelly', 'hypocrisy', 'mlk', 'mike', 'chipotle', 'breaking', 'sonics', 'murray', 'drunk', 'loyal', 'jolla', 'commentary', 'neighborhood', 'feedback', 'hyperbole', 'commenting', 'attempting', 'smoke', 'angeles', 'dictator', 'budget', 'goverment', 'referencing', 'cause', 'picture', 'isos', 'bias', 'suitable', 'separated', 'reel', 'weird', 'offending', 'relies', 'strip', 'appease', 'texture', 'upvotes', 'space', 'finishing', 'association', 'extraordinarily', 'lightning', 'turnover', 'turning', 'evenly', 'universally', 'bloody', 'fmvp', 'organic', 'fertitta', 'streamable', 'young', 'minded', 'expiring', 'haircut', 'tie', 'gsw', 'http', 'petty', 'software', 'troop', 'supportive', 'simply', 'prone', 'paired', 'design', 'respectable', 'fat', 'downvoted', 'insurance', 'frickin', 'skinny', 'decides', 'oversized', 'hahahaha', 'stealing', 'mission', 'bombing', 'driven', 'truth', 'measure', 'sino', 'misconstrued', 'drexler', 'targeted', 'rebounding', 'franchise', 'serve', 'liking', 'search', 'half', 'pause', 'reactionary', 'oppressor', 'fundamentally', 'pressure', 'corporate', 'online', 'drove', 'cracking', 'handle', 'responsible', 'thoroughly', 'noticing', 'dun', 'illusion', 'flirting', 'ofc', 'thereof', 'distinctly', 'generic', 'deflection', 'swaggy', 'window', 'duncan', 'fewer', 'embraced', 'closet', 'page', 'flair', 'packing', 'buried', 'mask', 'stripper', 'slightly', 'reek', 'tweet', 'reality', 'layup', 'booty', 'brigaded', 'changing', 'kick', 'string', 'allen', 'mwp', 'script', 'ala', 'learning', 'invest', 'persecuting', 'understanding', 'commenter', 'dmorey', 'posting', 'flexing', 'circus', 'gene', 'december', 'surrounded', 'prof', 'handling', 'saved', 'quoting', 'imply', 'soil', 'determination', 'kike', 'protection', 'seat', 'written', 'cruel', 'mentor', 'fallen', 'flame', 'typical', 'creates', 'allows', 'craig', 'free', 'penny', 'baller', 'teammate', 'gold', 'providing', 'treat', 'presumably', 'freshest', 'requires', 'inflict', 'asylum', 'monitor', 'going', 'carry', 'obsessed', 'cult', 'wipe', 'lmaoo', 'separation', 'panty', 'alright', 'tantrum', 'chicago', 'cite', 'fine', 'roughly', 'accurately', 'brace', 'shumpert', 'meeting', 'raising', 'midseason', 'staying', 'jpeg', 'funniest', 'sworn', 'wider', 'risk', 'shouldnt', 'viral', 'waste', 'oklahoma', 'flying', 'account', 'clarify', 'disgust', 'beneficial', 'overrating', 'incapable', 'ancient', 'smart', 'discounted', 'press', 'feel', 'habit', 'gov', 'unprofessional', 'mclemore', 'dare', 'legend', 'follows', 'mandarin', 'dominance', 'iirc', 'heading', 'plummet', 'intended', 'ninja', 'game', 'logo', 'bush', 'handler', 'suspicious', 'sounded', 'crafting', 'burner', 'suicide', 'steve', 'revolution', 'offseason', 'sneak', 'taco', 'alexander', 'spoke', 'counter', 'korea', 'wasteland', 'issue', 'prigioni', 'brotherhood', 'awesome', 'rampant', 'time', 'valid', 'cheddar', 'feed', 'swole', 'pull', 'economy', 'boner', 'brick', 'kuz', 'accountable', 'silent', 'boo', 'lap', 'upcoming', 'disaster', 'cop', 'blessing', 'acquired', 'complain', 'honest', 'fascist', 'trailblazer', 'authoritarian', 'paid', 'report', 'occurred', 'faster', 'pussy', 'legacy', 'battle', 'backwards', 'bearing', 'republican', 'rely', 'number', 'translate', 'sits', 'reputation', 'bathroom', 'tonight', 'laugh', 'mustard', 'trash', 'automatic', 'donut', 'ppg', 'hof', 'bringing', 'legitimately', 'elite', 'revealed', 'teacher', 'spamming', 'harvested', 'catapult', 'fold', 'checking', 'rmb', 'hell', 'liquid', 'creepy', 'college', 'offering', 'seal', 'lady', 'drop', 'insane', 'advanced', 'fiasco', 'hating', 'error', 'basic', 'twist', 'nationwide', 'powerful', 'understandable', 'crop', 'soundbite', 'proud', 'argue', 'dress', 'racial', 'cover', 'magic', 'heck', 'author', 'fund', 'margin', 'lotta', 'lifelong', 'tweeted', 'stomp', 'figured', 'analytical', 'jumper', 'viewer', 'james', 'honorable', 'tbh', 'democracy', 'contributing', 'worded', 'grocery', 'baffled', 'wedding', 'rack', 'awful', 'buffalo', 'century', 'torture', 'dennis', 'committed', 'guessing', 'humanity', 'cringey', 'curse', 'jamal', 'working', 'outspoken', 'fouled', 'stockton', 'court', 'chanting', 'dragged', 'role', 'celebrity', 'thanks', 'thread', 'nowadays', 'popular', 'silver', 'shallow', 'ejected', 'broke', 'lil', 'quietly', 'garbage', 'dolan', 'remarkable', 'worldwide', 'finance', 'social', 'teen', 'platinum', 'misinformed', 'rarely', 'revolutionized', 'pocket', 'earth', 'muh', 'globalization', 'grade', 'straw', 'macho', 'tactic', 'guard', 'yeah', 'day', 'euro', 'banning', 'weapon', 'stopping', 'technically', 'receiving', 'chicken', 'wishing', 'andrew', 'holiday', 'little', 'animal', 'chandler', 'asia', 'wednesday', 'kowtow', 'department', 'according', 'loving', 'imma', 'default', 'informed', 'lover', 'smh', 'hominem', 'pete', 'dogshit', 'threatened', 'challenged', 'lonzo', 'junior', 'trust', 'claiming', 'helping', 'recover', 'trophy', 'counted', 'critic', 'concrete', 'spain', 'brown', 'overlooked', 'kerr', 'implementing', 'epic', 'generally', 'motion', 'reset', 'derailed', 'nail', 'camera', 'open', 'baylor', 'argument', 'everyday', 'assassinated', 'adam', 'fortunate', 'original', 'struck', 'nsfw', 'kareem', 'extremely', 'manufacturing', 'bennet', 'detention', 'genetics', 'goosebump', 'east', 'visited', 'forgiven', 'behavior', 'ded', 'preserving', 'sensitivity', 'met', 'signing', 'trez', 'bumped', 'bite', 'deported', 'hammy', 'hold', 'prediction', 'houstonrockets', 'josh', 'technical', 'hooker', 'sudden', 'motherfucking', 'ground', 'user', 'street', 'fetch', 'consequence', 'treated', 'projected', 'verify', 'cherry', 'sportsnet', 'applies', 'retired', 'uneducated', 'institution', 'playstyle', 'tamed', 'cotton', 'random', 'sector', 'market', 'maintain', 'ted', 'associated', 'international', 'sanity', 'distinction', 'sorry', 'weigh', 'trying', 'reaching', 'parson', 'glimpse', 'organization', 'gettin', 'shitting', 'amazon', 'deadline', 'dictate', 'felton', 'character', 'cow', 'pro', 'drag', 'ayton', 'seemingly', 'uighur', 'injection', 'mother', 'final', 'varying', 'goat', 'chill', 'idea', 'branding', 'barton', 'superhero', 'feeling', 'geopolitical', 'potential', 'shame', 'cowardice', 'adjust', 'compete', 'heated', 'dishonest', 'ticking', 'machine', 'pacer', 'despicable', 'pdf', 'chinese', 'sub', 'difficult', 'apparent', 'mocking', 'stare', 'yankee', 'tryna', 'witness', 'picked', 'perceived', 'appear', 'toy', 'redd', 'pel', 'commentmessage', 'new', 'seed', 'deny', 'contention', 'pleased', 'puppet', 'nbafullhd', 'matchup', 'smfh', 'toe', 'funding', 'israel', 'summed', 'soundtrack', 'eject', 'miller', 'comeback', 'today', 'subscribe', 'risking', 'followed', 'worry', 'thomas', 'exert', 'meaningless', 'prove', 'oppressed', 'felt', 'radical', 'footage', 'right', 'newspaper', 'suggest', 'contender', 'precaution', 'mascot', 'isolation', 'tour', 'lump', 'commissioner', 'gate', 'hker', 'regime', 'donate', 'arab', 'champ', 'use', 'bodied', 'remind', 'triggered', 'universal', 'officially', 'disgusting', 'exit', 'iso', 'armstrong', 'ex', 'ryan', 'closing', 'deserving', 'easiest', 'punished', 'apg', 'overreacting', 'vocal', 'acknowledged', 'disillusioned', 'dominated', 'provides', 'unarmed', 'living', 'handedly', 'resolution', 'longer', 'tuesday', 'assumption', 'bobby', 'wondering', 'driving', 'exposed', 'shopped', 'boogeyman', 'song', 'upset', 'bench', 'voluntarily', 'explained', 'mock', 'texan', 'brutally', 'presti', 'aboard', 'hush', 'pink', 'noticed', 'declared', 'ill', 'massive', 'nonsense', 'guess', 'mcl', 'willed', 'learn', 'unit', 'justify', 'category', 'publish', 'remotely', 'movement', 'runnin', 'calibre', 'siding', 'auto', 'whisperer', 'nike', 'vega', 'incoming', 'tilmanjfertitta', 'sudan', 'totalitarianism', 'oregon', 'vast', 'reminder', 'replaced', 'intl', 'punk', 'dissident', 'png', 'cloud', 'cyber', 'economically', 'water', 'vein', 'mum', 'repercussion', 'asked', 'youtu', 'independence', 'matured', 'streaming', 'shopping', 'measured', 'boost', 'copied', 'extradited', 'voted', 'solid', 'pack', 'follow', 'att', 'starting', 'hat', 'mobile', 'murderous', 'clamp', 'combo', 'xinjiang', 'long', 'settled', 'multiple', 'reason', 'renewed', 'bzdelik', 'preferred', 'cell', 'operated', 'pulling', 'die', 'smelling', 'liked', 'harvard', 'protected', 'finish', 'laid', 'player', 'beast', 'february', 'anderson', 'genuinely', 'mod', 'impressive', 'drone', 'text', 'hitler', 'case', 'african', 'required', 'crime', 'welp', 'planet', 'booing', 'pool', 'rocky', 'streamed', 'shaping', 'split', 'braindead', 'jordan', 'dirk', 'desperate', 'beverley', 'nice', 'spreading', 'expressing', 'listed', 'honor', 'robinson', 'badass', 'larger', 'central', 'dozen', 'instance', 'holding', 'irving', 'hide', 'energy', 'cast', 'consistent', 'insight', 'jailed', 'costume', 'initial', 'justified', 'dumpster', 'backtrack', 'theater', 'siakam', 'knight', 'satisfy', 'forcibly', 'zero', 'undrafted', 'saying', 'wonder', 'platform', 'plotting', 'tax', 'iphone', 'lived', 'warmups', 'delicious', 'decided', 'reunification', 'bare', 'martyr', 'afternoon', 'stephen', 'moved', 'purpose', 'thinking', 'capela', 'extremist', 'funny', 'hello', 'philadelphia', 'inferior', 'kurd', 'month', 'bubble', 'cage', 'paraphrasing', 'disney', 'antoni', 'standing', 'movie', 'montiejunas', 'faith', 'defensive', 'dealt', 'dante', 'bar', 'chop', 'hospital', 'stem', 'structure', 'traveling', 'lou', 'lay', 'expected', 'bpm', 'employer', 'uneasy', 'drafted', 'gentry', 'expert', 'racism', 'representing', 'suffered', 'resolve', 'bandwagon', 'differently', 'academia', 'efficiency', 'photoshops', 'care', 'england', 'ignore', 'dream', 'dissenting', 'troll', 'class', 'bitter', 'occasionally', 'pretty', 'freaking', 'prob', 'iron', 'vacuum', 'illegal', 'envy', 'pic', 'supporter', 'existed', 'gleague', 'brand', 'overpriced', 'turned', 'stale', 'approaching', 'closely', 'mainland', 'partisan', 'sneeze', 'uninformed', 'rainbow', 'average', 'truthfully', 'dickhead', 'max', 'commitment', 'stability', 'science', 'expression', 'seeking', 'subsequent', 'speaking', 'disappointed', 'rice', 'safety', 'tomorrow', 'previously', 'typically', 'ride', 'ummm', 'danger', 'hung', 'assignment', 'flaw', 'bone', 'hypocrite', 'facing', 'prisoner', 'passion', 'relax', 'underdog', 'weekend', 'aimlessly', 'subsidized', 'hahahahaha', 'creating', 'persona', 'anybody', 'awareness', 'negotiation', 'beneath', 'restaurant', 'louisville', 'asb', 'liberal', 'silencing', 'room', 'dig', 'realistically', 'resigned', 'kind', 'conley', 'ingles', 'color', 'starter', 'moron', 'jfc', 'sarver', 'suppressing', 'denying', 'continent', 'expensive', 'proportion', 'trim', 'masterful', 'needed', 'allll', 'thx', 'summer', 'patriot', 'laker', 'stronger', 'pretending', 'diet', 'war', 'recognition', 'grizzly', 'ruling', 'evolving', 'explain', 'affirm', 'drawing', 'sacrifice', 'cool', 'death', 'poorer', 'save', 'pb', 'spicy', 'universe', 'offensive', 'western', 'likely', 'stadium', 'scoring', 'opponent', 'tempo', 'insightful', 'entry', 'sponsored', 'circumstance', 'oppression', 'client', 'competent', 'sort', 'spell', 'kelvin', 'said', 'readily', 'disrespected', 'arguably', 'brainwashing', 'break', 'ruining', 'clicking', 'neighbor', 'shitstorm', 'way', 'believing', 'opening', 'bev', 'ngl', 'performance', 'bullied', 'harm', 'shock', 'horse', 'nbatv', 'big', 'jersey', 'access', 'zach', 'freshman', 'leverage', 'vet', 'lebrons', 'lemonade', 'impending', 'recently', 'properly', 'sale', 'completely', 'locked', 'dedicated', 'grip', 'judgment', 'asap', 'faking', 'motivated', 'athlete', 'whats', 'blackout', 'heaven', 'speculating', 'committing', 'religion', 'build', 'kyrie', 'minute', 'closed', 'possibly', 'monster', 'buy', 'gained', 'look', 'helluva', 'style', 'reacted', 'characteristic', 'cut', 'screened', 'censor', 'confidence', 'girlfriend', 'philly', 'ban', 'equally', 'graduate', 'gotcha', 'parade', 'soon', 'camo', 'infinite', 'separate', 'ideally', 'ken', 'greek', 'rate', 'forget', 'euroleague', 'false', 'bradley', 'slaughtered', 'steph', 'linsanity', 'undoubtedly', 'dallas', 'shit', 'worm', 'placard', 'swapped', 'possession', 'shadow', 'bum', 'training', 'trading', 'kill', 'obligation', 'fuck', 'light', 'stream', 'thumb', 'carrying', 'ducking', 'szn', 'foster', 'refusal', 'resemblance', 'consult', 'routinely', 'diva', 'require', 'operation', 'blatant', 'accepted', 'dear', 'covering', 'coaching', 'muslim', 'rise', 'music', 'advantage', 'origin', 'similarly', 'cough', 'socialist', 'receive', 'rare', 'embiid', 'comfortable', 'hield', 'erase', 'pay', 'election', 'scenario', 'goodnight', 'swung', 'deadly', 'dictatorial', 'tldr', 'lasting', 'bomb', 'reference', 'eyeball', 'communication', 'meant', 'versa', 'buddy', 'master', 'soft', 'attempted', 'south', 'athleticism', 'scrub', 'forgive', 'houston', 'bois', 'grave', 'uncle', 'walking', 'rhythm', 'duck', 'reffed', 'muscle', 'discourse', 'dumber', 'village', 'suck', 'art', 'massively', 'justice', 'foil', 'hanging', 'worst', 'civil', 'lucky', 'succumb', 'drastically', 'shuts', 'shut', 'lavine', 'luster', 'promotion', 'seen', 'gif', 'statement', 'villain', 'orwellian', 'shilling', 'kiss', 'blame', 'utter', 'luxury', 'exploded', 'shipping', 'joy', 'problem', 'deflecting', 'acl', 'deng', 'taiwan', 'wood', 'check', 'huh', 'lifetime', 'program', 'venezuela', 'chooses', 'tip', 'jeff', 'yr', 'exposing', 'kashmir', 'mit', 'logical', 'fraction', 'bored', 'boxer', 'chamberlain', 'vice', 'oscar', 'sink', 'disparity', 'skinned', 'analysis', 'announcer', 'uyghurs', 'expendable', 'smack', 'native', 'horrible', 'tegridy', 'nephew', 'genocide', 'measurement', 'disregard', 'favor', 'hawaiian', 'memory', 'immaculate', 'jacket', 'champion', 'wear', 'infrastructure', 'frank', 'raise', 'bullshit', 'ideal', 'david', 'rico', 'share', 'ceded', 'francisco', 'camp', 'roll', 'cooler', 'ink', 'coffee', 'shill', 'shareutm', 'lanky', 'victim', 'continuing', 'sick', 'network', 'meaning', 'criticize', 'bot', 'rift', 'alternative', 'thousand', 'solve', 'activity', 'respect', 'nfl', 'managed', 'succeeding', 'violently', 'child', 'jumbotron', 'crystal', 'tend', 'hope', 'york', 'video', 'wise', 'greatest', 'favour', 'trigger', 'wound', 'jack', 'dawg', 'messed', 'lamar', 'peer', 'party', 'jared', 'stated', 'embarrassed', 'glorious', 'ability', 'win', 'plz', 'chasing', 'void', 'city', 'sand', 'exists', 'talent', 'listing', 'response', 'nigga', 'richest', 'drip', 'comfortably', 'serviceable', 'arrived', 'feasting', 'advocating', 'blown', 'deserve', 'talked', 'dreadlock', 'dame', 'injustice', 'mentioned', 'rebel', 'toeing', 'irrelevant', 'tortured', 'reward', 'ranking', 'atrocity', 'annoying', 'dominating', 'twice', 'censoring', 'trio', 'block', 'bystander', 'beat', 'perform', 'progress', 'engaged', 'maker', 'htm', 'ft', 'handing', 'resting', 'loved', 'rgv', 'tracking', 'gtfo', 'leaving', 'till', 'pie', 'purposefully', 'autonomy', 'turner', 'paper', 'don', 'lakers', 'chant', 'bird', 'unhappy', 'payne', 'coupled', 'associate', 'defenseless', 'feature', 'porzingod', 'protecting', 'brings', 'realize', 'adjusted', 'accept', 'underestimating', 'blue', 'taking', 'jail', 'suspension', 'lefty', 'celtic', 'cuz', 'gymnastics', 'touch', 'pop', 'course', 'impact', 'suddenly', 'hair', 'elbow', 'culture', 'sense', 'material', 'memed', 'tank', 'tape', 'evaluation', 'seeing', 'bueno', 'wit', 'postseason', 'punch', 'recommend', 'warfare', 'freedom', 'interviewer', 'cardio', 'outside', 'convince', 'dictionary', 'school', 'god', 'mama', 'locking', 'eddie', 'redditor', 'oligarch', 'sock', 'perfectly', 'btw', 'hit', 'ring', 'code', 'unbelievable', 'admit', 'journey', 'close', 'stepping', 'shutting', 'evan', 'passed', 'asleep', 'updated', 'borderline', 'delete', 'film', 'inherently', 'kyle', 'combine', 'physically', 'taller', 'portland', 'wouldn', 'oldest', 'couch', 'wanting', 'concerned', 'attach', 'popovich', 'hmmmm', 'calculus', 'entity', 'trade', 'bitch', 'protestors', 'influx', 'action', 'scale', 'crucial', 'forgot', 'dickish', 'disgusted', 'running', 'shake', 'couldn', 'door', 'questioned', 'chump', 'morale', 'allow', 'struggling', 'investing', 'height', 'seriously', 'invested', 'bucket', 'equivalent', 'toss', 'caught', 'competition', 'oblivious', 'dinosaur', 'curious', 'flight', 'cutting', 'fail', 'baby', 'consensus', 'corny', 'confirmed', 'desired', 'fixed', 'ensure', 'kool', 'impactful', 'academy', 'clue', 'fuel', 'resistance', 'punish', 'clothes', 'robert', 'turkestan', 'multitude', 'enjoyed', 'challenge', 'sprain', 'retained', 'gimmick', 'stephs', 'isnt', 'ross', 'special', 'yellow', 'coach', 'spine', 'whining', 'selection', 'praised', 'extra', 'yah', 'olajuwon', 'ahead', 'pan', 'uploads', 'hitting', 'abc', 'barkley', 'jesus', 'chink', 'trevor', 'overwhelming', 'morning', 'hired', 'degree', 'absence', 'american', 'diamond', 'bowl', 'shirt', 'outdoor', 'rando', 'micro', 'substantially', 'yemen', 'hug', 'john', 'relieve', 'assert', 'cheer', 'matched', 'insecure', 'robertson', 'jim', 'traffic', 'intent', 'backboard', 'uhhh', 'elevate', 'merit', 'wife', 'stand', 'initially', 'syndrome', 'united', 'point', 'gap', 'scene', 'evident', 'bell', 'territory', 'discus', 'encourages', 'notion', 'contribute', 'rip', 'begin', 'belt', 'crackdown', 'sport', 'intimate', 'sanction', 'antonio', 'nah', 'stifling', 'rose', 'spaced', 'cheap', 'merchandise', 'governor', 'nationalistic', 'dolla', 'nickname', 'march', 'timed', 'represent', 'smaller', 'seriousness', 'corruption', 'transaction', 'hand', 'cooked', 'rest', 'dial', 'table', 'totalitarian', 'watch', 'immediate', 'mismatch', 'lethal', 'nba', 'cringy', 'ethnic', 'usher', 'focusing', 'olympics', 'friendly', 'overachieving', 'knicks', 'difference', 'gunna', 'sour', 'explosive', 'millionaire', 'mongolian', 'canceled', 'moses', 'rebounder', 'implication', 'race', 'wikipedia', 'product', 'android', 'parity', 'jon', 'entitled', 'girl', 'approve', 'diehard', 'son', 'model', 'squad', 'public', 'retain', 'turd', 'sing', 'shine', 'excited', 'courtesy', 'meter', 'bleach', 'upvoted', 'witnessed', 'blasting', 'torched', 'nugget', 'force', 'spam', 'textbook', 'concussion', 'putting', 'mass', 'malone', 'moving', 'improvement', 'consideration', 'email', 'grass', 'fuckin', 'winnie', 'gon', 'shamble', 'principled', 'submission', 'victory', 'considered', 'decency', 'dump', 'rodney', 'ironically', 'tanking', 'indiana', 'somebody', 'amazing', 'wrong', 'designed', 'oddly', 'generational', 'price', 'minus', 'indicator', 'upbringing', 'parroted', 'concentration', 'publicity', 'developed', 'desert', 'insanely', 'aim', 'improving', 'father', 'embrace', 'juice', 'sixer', 'ridiculous', 'push', 'bed', 'overrated', 'badge', 'got', 'useless', 'gist', 'threw', 'burning', 'loaded', 'stage', 'plain', 'general', 'narrative', 'consecutive', 'sat', 'meantime', 'creativity', 'chunk', 'cpa', 'ran', 'specify', 'smoother', 'passionate', 'fga', 'field', 'able', 'speaker', 'partially', 'generation', 'engineer', 'walked', 'tunnel', 'downplaying', 'infamous', 'duo', 'hefty', 'live', 'overlook', 'wcsf', 'near', 'writing', 'contribution', 'disappointment', 'mere', 'roberson', 'direct', 'kanye', 'condition', 'iggy', 'spiral', 'age', 'golden', 'losing', 'rounded', 'contentious', 'superior', 'lmfaoo', 'capitalize', 'southwest', 'unaware', 'bigotry', 'gerald', 'understatement', 'home', 'initiative', 'facebook', 'poster', 'jokic', 'ronald', 'decide', 'pairing', 'gary', 'beal', 'large', 'dropping', 'internet', 'immoral', 'projection', 'worthwhile', 'hard', 'lookin', 'range', 'desire', 'utah', 'jpg', 'peak', 'anta', 'globe', 'bookstore', 'spot', 'ultimately', 'rebuild', 'wildly', 'element', 'centrist', 'assured', 'exercising', 'slot', 'apologizing', 'unfair', 'midranges', 'chris', 'succeeds', 'hilarious', 'ticket', 'choose', 'texting', 'advice', 'installed', 'key', 'constantly', 'benefit', 'morey', 'motto', 'certified', 'balling', 'amid', 'voter', 'dafuq', 'concession', 'hardly', 'percent', 'car', 'apologizes', 'prime', 'sucking', 'sadly', 'current', 'grand', 'hmm', 'gladly', 'exec', 'cat', 'sheesh', 'literally', 'sniffed', 'bennett', 'posted', 'shrug', 'blatantly', 'soldier', 'unfortunately', 'attendance', 'communism', 'crossing', 'na', 'dangerous', 'brb', 'straight', 'manager', 'george', 'using', 'aremindme', 'sacramento', 'happening', 'sim', 'mad', 'carolina', 'maga', 'impressed', 'unload', 'determined', 'population', 'wether', 'dug', 'prop', 'growing', 'reserve', 'numerous', 'wide', 'date', 'lighting', 'tall', 'chino', 'spending', 'boil', 'inexplicable', 'capable', 'resume', 'valuation', 'kansa', 'drum', 'covington', 'president', 'tnt', 'hockey', 'fraught', 'madness', 'joseph', 'remark', 'roger', 'tail', 'joke', 'happier', 'continued', 'fond', 'sarcastic', 'thompson', 'fit', 'hobby', 'boogie', 'discovery', 'fly', 'filter', 'hyped', 'clickbait', 'secede', 'existing', 'blank', 'trainer', 'stepback', 'custom', 'domestic', 'play', 'sweat', 'threatening', 'bogut', 'cracked', 'physical', 'darryl', 'kindly', 'climb', 'interestingly', 'reddick', 'supposedly', 'terry', 'tier', 'probable', 'bleacher', 'spacex', 'journalist', 'ship', 'stance', 'racked', 'affecting', 'playable', 'voice', 'pelican', 'pirate', 'falling', 'collect', 'citizen', 'spouting', 'winded', 'indicative', 'meme', 'stepped', 'terrifying', 'puma', 'ad', 'watching', 'goodness', 'cried', 'publicly', 'brush', 'crappy', 'pnr', 'extraordinary', 'superiority', 'commute', 'posse', 'waving', 'clash', 'speaks', 'le', 'rus', 'rim', 'shaq', 'ml', 'born', 'boycotting', 'lower', 'base', 'revenue', 'killed', 'ussr', 'russia', 'viable', 'alvin', 'threshold', 'main', 'quotation', 'explains', 'livid', 'news', 'shoe', 'refuse', 'investment', 'minnesota', 'community', 'represents', 'requirement', 'engagement', 'security', 'familiar', 'jones', 'easier', 'chance', 'analyst', 'signaling', 'known', 'standpoint', 'clever', 'try', 'white', 'lunch', 'apputm', 'brighter', 'maintained', 'prayer', 'america', 'knocking', 'gobert', 'mouth', 'yank', 'backup', 'leadership', 'coward', 'reasoned', 'clear', 'pepper', 'idiotic', 'mix', 'fortunately', 'navy', 'fundamental', 'chilling', 'crack', 'club', 'rebuilding', 'waking', 'place', 'pace', 'computer', 'suffering', 'fanboy', 'businessinsider', 'charles', 'remindersmessage', 'flash', 'iverson', 'housing', 'plot', 'essay', 'quicker', 'presidential', 'complaining', 'confirmation', 'lmao', 'hopefully', 'korean', 'voicing', 'censorship', 'manage', 'igshid', 'soviet', 'naw', 'determine', 'regain', 'zealand', 'blaming', 'darn', 'bastard', 'dominant', 'reading', 'bowling', 'canada', 'wave', 'grew', 'kongers', 'countryman', 'hypothetically', 'resource', 'realm', 'address', 'rap', 'future', 'active', 'com', 'weibo', 'labor', 'launch', 'dunk', 'episode', 'subscription', 'ene', 'spare', 'survive', 'problematic', 'amp', 'jab', 'unless', 'support', 'baseball', 'speech', 'deliberately', 'album', 'motherfucker', 'budenholzer', 'prize', 'football', 'bonus', 'sam', 'worried', 'quit', 'exclusive', 'regularly', 'criminal', 'fairly', 'parking', 'view', 'tension', 'destroyed', 'tuning', 'silence', 'ass', 'react', 'dad', 'johnson', 'expand', 'bankrupt', 'complaint', 'period', 'southern', 'zion', 'sooner', 'older', 'theoretically', 'oops', 'equating', 'lesson', 'bag', 'eaten', 'php', 'reached', 'sustain', 'imprison', 'rioting', 'jackie', 'parent', 'snitch', 'guaranteed', 'idiot', 'anomaly', 'floating', 'spilling', 'whore', 'strategy', 'lawyer', 'partner', 'pressured', 'appropriately', 'process', 'spalding', 'paint', 'gross', 'collapsed', 'lol', 'maverick', 'screaming', 'moral', 'blazer', 'apologise', 'imagine', 'realized', 'staple', 'label', 'outcome', 'remember', 'surprise', 'making', 'sharing', 'ahhh', 'quoque', 'skin', 'buying', 'connected', 'overall', 'shade', 'looking', 'releasing', 'basis', 'undeniable', 'handed', 'ish', 'klay', 'fleeing', 'imgur', 'accomplishes', 'bullet', 'disingenuous', 'frustrated', 'willingly', 'controlled', 'invasion', 'lazy', 'jinping', 'house', 'cba', 'yay', 'indifferent', 'experienced', 'hurting', 'reply', 'doin', 'pride', 'sleeping', 'rosy', 'fancy', 'pant', 'shifting', 'centre', 'medical', 'integrity', 'flop', 'addressing', 'anti', 'people', 'condone', 'pitchfork', 'sponsorship', 'member', 'mexican', 'pacific', 'betrayed', 'owe', 'emotion', 'matching', 'username', 'wished', 'mediocre', 'aside', 'commentator', 'halfcourt', 'basketball', 'ymca', 'friday', 'tool', 'influencing', 'shouldn', 'occasion', 'filipino', 'sentence', 'ssol', 'tankie', 'replacing', 'nerd', 'hey', 'pathetic', 'additional', 'thursday', 'discord', 'pile', 'hiring', 'anniversary', 'custody', 'showed', 'haha', 'meet', 'youre', 'nationalism', 'choosing', 'turn', 'recognize', 'sec', 'figuring', 'growth', 'resort', 'just', 'administration', 'night', 'pick', 'fool', 'qualified', 'harvesting', 'winner', 'whack', 'critically', 'ups', 'instead', 'bet', 'kobe', 'temporary', 'tolerated', 'hopeful', 'friend', 'analytics', 'accuse', 'iguodala', 'innocent', 'attached', 'add', 'old', 'livelihood', 'profitable', 'amendment', 'suggesting', 'donkey', 'silly', 'cancel', 'killer', 'hall', 'lmaooooo', 'ready', 'inspirational', 'cruz', 'scandal', 'spoken', 'picking', 'popularity', 'tebow', 'screen', 'dreaming', 'keeping', 'dropoff', 'massacre', 'mesh', 'narrator', 'dodge', 'stuck', 'considering', 'philosophy', 'follower', 'box', 'mess', 'sell', 'lemme', 'sicko', 'survivor', 'legit', 'painful', 'shitposting', 'didn', 'executive', 'benevolent', 'gooo', 'headline', 'vest', 'averaged', 'form', 'punched', 'sway', 'available', 'chronic', 'unlimited', 'hardcore', 'faker', 'proven', 'contest', 'october', 'roster', 'playmaker', 'article', 'foh', 'plus', 'wed', 'yesterday', 'shanghai', 'expose', 'grab', 'option', 'decker', 'efg', 'wiggins', 'ousted', 'understands', 'purchasing', 'deeper', 'criticizing', 'lighter', 'strongest', 'hol', 'egregious', 'heavily', 'outlet', 'baseless', 'definitely', 'outline', 'headband', 'repeat', 'blind', 'debt', 'hongkong', 'mavs', 'sky', 'focus', 'pioneer', 'surface', 'talented', 'suspect', 'twitter', 'bat', 'mirror', 'ankle', 'think', 'privy', 'smug', 'deceptive', 'escalate', 'restraint', 'redacted', 'norm', 'privilege', 'tad', 'organ', 'kid', 'europe', 'terrible', 'privileged', 'whiteside', 'compromise', 'fansided', 'heart', 'stood', 'abroad', 'wouldnt', 'behalf', 'interview', 'order', 'rugby', 'sharp', 'albeit', 'broken', 'existence', 'whatsoever', 'slowly', 'hang', 'versed', 'mon', 'king', 'articulated', 'relevant', 'joining', 'pressing', 'genius', 'knocked', 'scrolling', 'coverage', 'pass', 'injury', 'irony', 'largest', 'compelled', 'manu', 'drawn', 'noise', 'tear', 'dodger', 'priced', 'involve', 'anymore', 'plugged', 'british', 'prince', 'professor', 'exist', 'inflammatory', 'obviously', 'recovery', 'organisation', 'change', 'premium', 'scary', 'lesser', 'argued', 'goodbye', 'lastly', 'dumb', 'beater', 'soooo', 'homeland', 'slap', 'demonstrating', 'momentum', 'boneheaded', 'duty', 'dimensional', 'stuff', 'practically', 'insist', 'stating', 'needle', 'focused', 'baggage', 'difficulty', 'debatable', 'destroying', 'raptor', 'basically', 'boycott', 'website', 'imperialism', 'vietnam', 'inside', 'disagreeing', 'moderator', 'denounces', 'racist', 'rising', 'dealing', 'flat', 'postgame', 'ffs', 'aka', 'depending', 'exaggeration', 'entirely', 'insult', 'wet', 'belong', 'hehe', 'deandre', 'external', 'dark', 'bojan', 'accidentally', 'gong', 'cassell', 'soundbites', 'guarding', 'rent', 'tire', 'mentioning', 'fighting', 'cunt', 'clone', 'instructed', 'surprising', 'land', 'restore', 'positively', 'bull', 'doctor', 'tanked', 'morethananathlete', 'invite', 'blew', 'mom', 'honestly', 'includes', 'impose', 'permanent', 'nuanced', 'spur', 'dirty', 'fyi', 'bummer', 'butthurt', 'diminished', 'mpg', 'documentary', 'evidence', 'til', 'decent', 'spirit', 'granted', 'gamble', 'billionaire', 'opted', 'knock', 'comprehension', 'incorrect', 'aren', 'rapper', 'demand', 'psyched', 'frame', 'consume', 'discount', 'rekt', 'despise', 'bounce', 'atleast', 'impossible', 'motif', 'rotation', 'downvote', 'deemed', 'undesirable', 'hub', 'tilman', 'tix', 'healthy', 'kemp', 'blow', 'sandal', 'meh', 'position', 'channel', 'bruh', 'upload', 'flipped', 'waited', 'bow', 'significant', 'pretend', 'pollution', 'confident', 'limiting', 'crazy', 'dweeb', 'jeremy', 'respecting', 'tibetan', 'businessman', 'disappear', 'identity', 'kenny', 'fucked', 'fraud', 'summary', 'train', 'dime', 'fucker', 'streetwear', 'entering', 'mate', 'visiting', 'arena', 'policy', 'project', 'blm', 'tradition', 'possibility', 'sight', 'held', 'passing', 'specific', 'cheaper', 'lebron', 'essentially', 'proved', 'rude', 'progressive', 'equity', 'question', 'uber', 'given', 'apparel', 'dope', 'explode', 'draft', 'paul', 'reflection', 'translation', 'sounding', 'brodie', 'fighter', 'dysfunctional', 'appreciate', 'torn', 'childhood', 'po', 'integrated', 'left', 'youth', 'creative', 'low', 'objective', 'thought', 'lift', 'fruit', 'self', 'suit', 'caliber', 'framework', 'suggested', 'allowed', 'based', 'whale', 'idgaf', 'hear', 'ethnicity', 'senior', 'deer', 'version', 'greed', 'appearance', 'baba', 'gen', 'boom', 'creator', 'republic', 'sweep', 'opposite', 'tokyo', 'cushion', 'officer', 'end', 'oil', 'uniform', 'catalan', 'scholar', 'letter', 'bbiq', 'teach', 'politician', 'couple', 'offense', 'struggled', 'information', 'need', 'spotlight', 'card', 'oriented', 'retool', 'wasn', 'mic', 'arabia', 'interpreting', 'secret', 'ghost', 'shark', 'certain', 'received', 'sweet', 'partly', 'bait', 'glass', 'whatabout', 'apple', 'bos', 'despite', 'shameful', 'hop', 'deployment', 'hive', 'efficient', 'odd', 'smith', 'kawhi', 'tho', 'rebound', 'beach', 'sad', 'hoe', 'giannis', 'nate', 'steer', 'splashing', 'settle', 'passive', 'blood', 'willingness', 'crisis', 'barrel', 'potentially', 'police', 'widely', 'everytime', 'loose', 'factor', 'extradition', 'confusion', 'ball', 'taiwanese', 'classic', 'reporter', 'extrapolating', 'sponsor', 'company', 'manchester', 'okc', 'incredibly', 'farming', 'achieved', 'trezz', 'imgflip', 'clarification', 'area', 'sovereign', 'closest', 'knowing', 'possible', 'hurt', 'missed', 'marvin', 'singular', 'riot', 'truck', 'atrocious', 'owning', 'clothing', 'rotate', 'criticizes', 'begging', 'monk', 'imo', 'embassy', 'poor', 'shy', 'doubting', 'totally', 'inbox', 'mexico', 'saudi', 'understand', 'good', 'added', 'defender', 'blake', 'garner', 'insulting', 'mention', 'nmsl', 'hong', 'skate', 'capella', 'realistic', 'arsenal', 'crossed', 'forum', 'broadcasting', 'trend', 'studying', 'appearing', 'noooooo', 'finger', 'opinionated', 'whoop', 'surprised', 'yea', 'fashion', 'leading', 'outrage', 'mighty', 'altogether', 'appeal', 'worldnews', 'obligated', 'strength', 'ramification', 'holy', 'happen', 'mini', 'promised', 'promote', 'washington', 'superpower', 'know', 'cucked', 'setting', 'deep', 'denier', 'wack', 'pointless', 'eli', 'rivalry', 'televised', 'bald', 'detainee', 'rough', 'pal', 'arm', 'fist', 'adidas', 'rating', 'hollow', 'fanbase', 'mma', 'purse', 'catch', 'crisp', 'quality', 'amazingly', 'jonathan', 'stir', 'reasonably', 'outplayed', 'smarter', 'taylor', 'naive', 'operating', 'cent', 'load', 'buck', 'saw', 'earned', 'chain', 'enter', 'thunderous', 'anthem', 'arrogant', 'lbj', 'effectively', 'bogdanovic', 'cleansing', 'kinda', 'volume', 'inappropriate', 'los', 'value', 'suite', 'absolutely', 'enable', 'deeply', 'protects', 'addiction', 'loop', 'choker', 'qualifies', 'lot', 'rumor', 'entered', 'halftime', 'warren', 'dapping', 'retirement', 'favourite', 'produce', 'convention', 'convenient', 'disliked', 'erdo', 'clown', 'semblance', 'copyright', 'bother', 'tibet', 'experience', 'tenure', 'frankly', 'reminds', 'ccp', 'commie', 'paris', 'sentiment', 'beijing', 'temple', 'vulnerable', 'pray', 'lb', 'hype', 'useful', 'combined', 'guy', 'chat', 'purely', 'inevitable', 'dance', 'leaning', 'created', 'shift', 'crippled', 'bout', 'buyout', 'abuse', 'secure', 'raised', 'proxy', 'innovation', 'handful', 'conversation', 'absorbed', 'ultra', 'dressed', 'chuck', 'grown', 'branded', 'austin', 'story', 'inaccurate', 'slow', 'fellow', 'handled', 'yup', 'gasp', 'corner', 'phew', 'mark', 'anyways', 'liable', 'humiliated', 'tackle', 'westbrook', 'baron', 'student', 'glory', 'probability', 'patriotic', 'starving', 'consulate', 'jason', 'crap', 'round', 'similarity', 'memphis', 'reprimanded', 'gift', 'clutch', 'begun', 'unfortunate', 'removed', 'marcus', 'anticipation', 'economic', 'lob', 'brave', 'pasted', 'tone', 'huge', 'jeopardy', 'praise', 'cycle', 'jacking', 'extreme', 'responding', 'excel', 'especially', 'unreal', 'wumao', 'microphone', 'discussing', 'captain', 'spinning', 'oop', 'gut', 'prc', 'widespread', 'exploiting', 'chan', 'provocative', 'guessed', 'fact', 'phi', 'terrorism', 'underlying', 'strong', 'distanced', 'carefully', 'nick', 'foothold', 'individualism', 'disposable', 'viewpoint', 'shoot', 'purchase', 'perfect', 'url', 'law', 'solo', 'boutta', 'stifled', 'exploitation', 'banned', 'mean', 'theyre', 'asshat', 'kellerman', 'aligned', 'jedi', 'tyrannical', 'upgrade', 'acknowledging', 'obvious', 'strike', 'baseline', 'hnk', 'scared', 'denial', 'enjoying', 'weaker', 'awhile', 'unsure', 'played', 'phrase', 'claim', 'plant', 'unique', 'comprehend', 'heritage', 'blessed', 'endorsement', 'describing', 'messy', 'ohhh', 'theguardian', 'mcw', 'chained', 'estate', 'common', 'jan', 'gave', 'flag', 'ego', 'brought', 'seattle', 'damaging', 'needing', 'reign', 'roommate', 'rightfully', 'red', 'worrying', 'curry', 'switch', 'hahah', 'pendulum', 'legged', 'demanding', 'tencent', 'doesn', 'timeline', 'beauty', 'bad', 'suppress', 'simplistic', 'mentality', 'semi', 'connection', 'salary', 'incur', 'foreign', 'pointing', 'sideline', 'interior', 'pre', 'premier', 'job', 'riding', 'obliged', 'radio', 'acting', 'intentionally', 'increase', 'expects', 'watched', 'early', 'post', 'wuss', 'unnecessary', 'cleveland', 'heh', 'actor', 'trending', 'framed', 'defined', 'background', 'core', 'sir', 'reeducation', 'dominates', 'television', 'succeeded', 'privately', 'kdtrey', 'repeatedly', 'taste', 'level', 'formed', 'hoop', 'nation', 'perspective', 'depth', 'minority', 'twilight', 'woulda', 'arrest', 'subjective', 'normal', 'compose', 'agreeing', 'trial', 'fed', 'rogers', 'warm', 'refreshing', 'damn', 'pat', 'requiring', 'tried', 'apparently', 'advertiser', 'encouraging', 'wall', 'hypothetical', 'downvotes', 'cup', 'cowardly', 'internal', 'ignoring', 'leap', 'ordering', 'ingrained', 'slope', 'register', 'rule', 'gameplan', 'prevent', 'signed', 'scouting', 'vietnamese', 'giving', 'cowboy', 'neck', 'foreseeable', 'frequently', 'hill', 'plan', 'weakness', 'somewhat', 'communist', 'horizontal', 'relationship', 'adorable', 'developing', 'locker', 'vaguely', 'staff', 'blizzard', 'pursuit', 'reb', 'write', 'sunday', 'distracting', 'boomed', 'possesions', 'tangent', 'rolling', 'renew', 'bought', 'pond', 'crossover', 'brain', 'sound', 'masked', 'candy', 'comin', 'dim', 'equivalency', 'propaganda', 'terror', 'outrageous', 'beaten', 'snagged', 'joe', 'length', 'sandwich', 'toilet', 'concern', 'relatively', 'scare', 'particularly', 'wheel', 'costing', 'eat', 'additionally', 'attacked', 'ima', 'individual', 'double', 'slave', 'zubac', 'sister', 'tmac', 'compared', 'intellectually', 'burn', 'suggests', 'wcf', 'counting', 'history', 'mob', 'barry', 'ethic', 'footwork', 'beverly', 'attend', 'cnn', 'presence', 'texas', 'hot', 'did', 'ayyy', 'nap', 'aint', 'short', 'lord', 'built', 'penis', 'interested', 'breath', 'stroke', 'schedule', 'cake', 'kicking', 'speed', 'bigger', 'restriction', 'violent', 'tough', 'implied', 'married', 'tony', 'zone', 'offset', 'messing', 'promising', 'celebrating', 'flavor', 'appeasing', 'orange', 'assist', 'checked', 'english', 'smooth', 'relocate', 'declining', 'selling', 'afford', 'person', 'denouncing', 'pitch', 'rested', 'leader', 'beeing', 'ooof', 'folk', 'garage', 'tell', 'agenda', 'mil', 'gripe', 'hidden', 'martin', 'unpopular', 'abide', 'compassion', 'anthony', 'retro', 'demar', 'oakland', 'consider', 'aggressive', 'tired', 'piston', 'regained', 'display', 'vertical', 'moreyball', 'bin', 'struggle', 'lasted', 'runner', 'slowed', 'dem', 'opposition', 'commercial', 'lillard', 'roco', 'handover', 'australian', 'min', 'saint', 'island', 'ingredient', 'apps', 'bette', 'package', 'hogging', 'usual', 'traitor', 'fav', 'status', 'spin', 'stop', 'blamed', 'stat', 'overreaction', 'doc', 'separatist', 'willing', 'searching', 'waiting', 'commented', 'deleted', 'influential', 'payton', 'weren', 'mistaken', 'sickening', 'soccer', 'allowing', 'doing', 'oof', 'professionally', 'heat', 'entirety', 'christ', 'googled', 'conspiracy', 'lin', 'sweatshop', 'consumer', 'yessir', 'dignity', 'drink', 'management', 'ginobili', 'earnings', 'phone', 'rubio', 'unlike', 'doesnt', 'line', 'usa', 'analytically', 'feast', 'isn', 'situation', 'wow', 'demanded', 'fitting', 'delusional', 'skillset', 'nut', 'repost', 'phoenix', 'west', 'yang', 'everybody', 'landscape', 'loudly', 'paragraph', 'sportscenter', 'dunno', 'exercise', 'yeh', 'zeller', 'joking', 'basket', 'having', 'broadcast', 'drug', 'fantasy', 'gigantic', 'paycheck', 'employee', 'returning', 'prospect', 'tiananmen', 'wingspan', 'sensitive', 'promise', 'ridiculed', 'headed', 'swap', 'join', 'recorded', 'untouchable', 'trillion', 'contested', 'spiritual', 'peace', 'firmly', 'html', 'come', 'riled', 'walk', 'source', 'german', 'wtf', 'rocking', 'decrease', 'shape', 'concept', 'prevented', 'remembering', 'terrance', 'anecdotal', 'wish', 'thrown', 'belongs', 'bootlicker', 'nationality', 'brian', 'previous', 'debacle', 'prioritize', 'jump', 'pac', 'drill', 'mediocrity', 'record', 'questioning', 'intangible', 'bully', 'conservative', 'perimeter', 'commit', 'breakout', 'definite', 'ordered', 'encouraged', 'teenager', 'yes', 'homie', 'unproven', 'pair', 'executed', 'wallet', 'tiny', 'shooter', 'property', 'draw', 'suppression', 'vehicle', 'theory', 'wouldve', 'personal', 'avoided', 'fam', 'nostalgia', 'miami', 'turtle', 'drama', 'coulda', 'eligible', 'nearby', 'say', 'combination', 'historical', 'path', 'billy', 'region', 'clip', 'political', 'remove', 'prior', 'defended', 'downtown', 'laterally', 'roadblock', 'nature', 'spineless', 'liberty', 'input', 'offensively', 'manufactured', 'lineup', 'exposure', 'fish', 'traditional', 'addition', 'second', 'beating', 'la', 'happened', 'daddy', 'gay', 'autograph', 'causing', 'cap', 'edge', 'kanter', 'recent', 'signal', 'research', 'underrating', 'shithole', 'forfeit', 'season', 'arrive', 'bright', 'believe', 'defends', 'dantoni', 'particular', 'fully', 'update', 'dikembe', 'jazz', 'govt', 'harrell', 'data', 'fell', 'rosa', 'comparing', 'strictly', 'trump', 'productive', 'country', 'piece', 'quarter', 'brother', 'supply', 'peaceful', 'swear', 'abandoned', 'book', 'wa', 'worrisome', 'gifs', 'unguardable', 'verbally', 'type', 'sampson', 'replaceable', 'leonard', 'plenty', 'dead', 'logically', 'grasp', 'adequate', 'apathy', 'high', 'insider', 'gassed', 'interpretation', 'announced', 'netflix', 'help', 'mode', 'getting', 'away', 'rounder', 'timing', 'cricket', 'rooting', 'expectation', 'freeze', 'lean', 'server', 'weed', 'admire', 'service', 'ducked', 'devin', 'profile', 'supported', 'burned', 'griffin', 'assuming', 'haven', 'similar', 'boycotted', 'rigged', 'behaviour', 'unheard', 'disrespectful', 'covered', 'reasoning', 'dummy', 'army', 'affected', 'famous', 'selfish', 'acted', 'countless', 'instantly', 'crowd', 'browse', 'miracle', 'consuming', 'snap', 'retires', 'boy', 'fuk', 'severity', 'touching', 'different', 'whataboutism', 'corporation', 'entertainment', 'footer', 'greatly', 'casuals', 'germany', 'monopoly', 'doubling', 'reflects', 'thing', 'performs', 'decline', 'used', 'shouting', 'wind', 'downhill', 'abraham', 'patrick', 'abused', 'calvin', 'office', 'nuance']

In [92]:
# extending stopwords.words('english') to include words that are common in both subreddits

stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(union_words)

In [93]:
%store stopwords

Stored 'stopwords' (list)
